In [36]:
import pandas as pd
from datetime import datetime, date

def rename_columns(df):
    existing_columns = ['Viaje_Id', 'Usuario_Id', 'Genero', 'Anio_de_nacimiento', 'Inicio_del_viaje', 'Fin_del_viaje', 'Origen_Id', 'Destino_Id']
    new_columns = ['trip_id', 'user_id', 'gender', 'birthday', 'start_trip', 'end_trip', 'origin_id', 'destination_id']
    
    for idx, new_column in enumerate(new_columns):
        df[new_column] = df[existing_columns[idx]]
        del df[existing_columns[idx]]
        
def add_new_columns(df):
    now = datetime.now()
    current_year = now.year

    # calculate user's age and add it to dataframe
    df["age"] = current_year - df["birthday"]
    
    # calculate trip duration
    df['start_trip_date'] = pd.to_datetime(df['start_trip'], format='%d/%m/%y %H:%M')
    df['end_strip_date'] = pd.to_datetime(df['end_trip'], format='%d/%m/%y %H:%M')
    
    df["trip_duration_hours"] = (df["end_strip_date"] - df['start_trip_date']) / pd.Timedelta(1, 'h')
    df["trip_duration_hours"] = df["trip_duration_hours"].round(2)
    df["trip_duration_minutes"] = (df["end_strip_date"] - df['start_trip_date']) / pd.Timedelta(1, 'm')
    df["year"] = df.start_trip_date.dt.year
    
    del df['start_trip_date']
    del df['end_strip_date']

def add_origin_destination_nomenclature(df, nomenclature):
    temp_df = df
    
    for left_on in ['origin', 'destination']:    
        temp_df = temp_df.merge(nomenclature, left_on=f"{left_on}_id", right_on='id', suffixes=(None, f"_{left_on}"))

        temp_df[f"{left_on}_name"] = temp_df['name']
        temp_df[f"{left_on}_obcn"] = temp_df['obcn']
        temp_df[f"{left_on}_location"] = temp_df['location']
        temp_df[f"{left_on}_latitude"] = temp_df['latitude']
        temp_df[f"{left_on}_longitude"] = temp_df['longitude']
        temp_df[f"{left_on}_status"] = temp_df['status']


        del temp_df['id']
        del temp_df['name']
        del temp_df['obcn']
        del temp_df['location']
        del temp_df['latitude']
        del temp_df['longitude']
        del temp_df['status']

    return temp_df

def reorder_columns(df):
    columns_order = [
        'trip_id', 
        'user_id', 
        'gender', 
        'birthday', 
        'age',
        'start_trip', 
        'end_trip',
        'trip_duration_minutes',
        'trip_duration_hours',
        'year',
        'origin_id',
        'origin_name',
        'origin_obcn',
        'origin_location',
        'origin_latitude',
        'origin_longitude',
        'origin_status',
        'destination_id',
        'destination_name',
        'destination_obcn',
        'destination_location',
        'destination_latitude',
        'destination_longitude',
        'destination_status'
    ]
    
    return df.reindex(columns=columns_order)


data_years = ['2023','2022', '2021', '2020']
data_months = ['05']

file_prefix = 'datos_abiertos_'
nomenclature_file = 'nomenclatura_2023_05'

nomenclature_df = pd.read_csv(f"{nomenclature_file}.csv")

data_frames_to_merge = []

for year in data_years:
    for month in data_months:
        file_name = f"{file_prefix}{year}_{month}"
        
        mibici_df = pd.read_csv(f"{file_name}.csv")
        
        # preparing data
        rename_columns(mibici_df)
        add_new_columns(mibici_df)
        mibici_df = add_origin_destination_nomenclature(mibici_df, nomenclature_df)
        mibici_df = reorder_columns(mibici_df)
        
#         mibici_df.to_csv(f"output/{file_name}_new.csv", index=False)
        data_frames_to_merge.append(mibici_df)
        
final_df = pd.concat(data_frames_to_merge)
final_df.to_csv(f"mibici_{data_years[-2]}_{data_years[0]}.csv", index=False)

print(final_df.head())


    trip_id  user_id  ... destination_longitude  destination_status
0  28467098    70123  ...            -103.40134          IN_SERVICE
1  28653697  1515654  ...            -103.40134          IN_SERVICE
2  28654633   306600  ...            -103.40134          IN_SERVICE
3  28799016  1107631  ...            -103.40134          IN_SERVICE
4  28548492  1744633  ...            -103.40134          IN_SERVICE

[5 rows x 24 columns]
